ThinkDSP
This notebook contains solutions to exercises in Chapter 7: Discrete Fourier Transform

Copyright 2015 Allen Downey

License: Creative Commons Attribution 4.0 International

In [1]:
import numpy as np
PI2 = 2 * np.pi

Exercise 1
In this chapter, I showed how we can express the DFT and inverse DFT as matrix multiplications. These operations take time proportional to 
, where 
 is the length of the wave array. That is fast enough for many applications, but there is a faster algorithm, the Fast Fourier Transform (FFT), which takes time proportional to 
.

The key to the FFT is the Danielson-Lanczos lemma:


Where 
 is the 
th element of the DFT of 
; 
 is the even elements of 
, and 
 is the odd elements of 
.

This lemma suggests a recursive algorithm for the DFT:

Given a wave array, 
, split it into its even elements, 
, and its odd elements, 
.

Compute the DFT of 
 and 
 by making recursive calls.

Compute 
 for each value of 
 using the Danielson-Lanczos lemma.

For the base case of this recursion, you could wait until the length of 
 is 1. In that case, 
. Or if the length of 
 is sufficiently small, you could compute its DFT by matrix multiplication, possibly using a precomputed matrix.

Hint: I suggest you implement this algorithm incrementally by starting with a version that is not truly recursive. In Step 2, instead of making a recursive call, use dft or np.fft.fft. Get Step 3 working, and confirm that the results are consistent with the other implementations. Then add a base case and confirm that it works. Finally, replace Step 2 with recursive calls.

In [2]:
ys = [-0.5, 0.1, 0.7, -0.1]
hs = np.fft.fft(ys)
print(hs)

[ 0.2+0.j  -1.2-0.2j  0.2+0.j  -1.2+0.2j]


In [3]:
def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

In [7]:
hs2 = dft(ys)
np.sum(np.abs(hs - hs2))
print(hs2)

[ 0.2+0.00000000e+00j -1.2-2.00000000e-01j  0.2+1.95943488e-16j
 -1.2+2.00000000e-01j]


In [5]:
def fft_norec(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

In [8]:
hs3 = fft_norec(ys)
np.sum(np.abs(hs - hs3))
print(hs3)

[ 0.2+0.j  -1.2-0.2j  0.2+0.j  -1.2+0.2j]


In [9]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys
    
    He = fft(ys[::2])
    Ho = fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

In [10]:
hs4 = fft(ys)
np.sum(np.abs(hs - hs4))
print(hs4)

[ 0.2+0.j  -1.2-0.2j  0.2+0.j  -1.2+0.2j]
